In [29]:
import os
import torch 
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as f

import numpy as np
import pandas as pd
import glob
import cv2
import matplotlib.pyplot as plt 
import torchvision.transforms as T
import tqdm
from torch.utils.data import DataLoader, sampler, random_split
from dataLoader import BatchLoader
from loss_functions import ICE_loss
from models.ImgEnhanceNet import ImgEnhanceNet

import warnings
warnings.filterwarnings("ignore")

%load_ext autoreload
%autoreload 2

ImportError: cannot import name 'ToPILImage' from 'torchvision.transforms.functional' (/home/x0yong/.local/lib/python3.7/site-packages/torchvision/transforms/functional.py)

In [31]:
np.array([[0, -1], [0, 1]]).T
k1 = np.transpose(np.array([[0, 0], [-1, 1]], dtype=np.float32).reshape((2, 2, 1, 1)), [3, 2, 0, 1])
np.transpose(np.array([[-1,0,1]], dtype=np.float32).reshape((1,3,1,1)), [3,2,0,1])

array([[[[-1.,  0.,  1.]]]], dtype=float32)

In [35]:
np.array([[-1,8,0,-8,1]])

array([[-1,  8,  0, -8,  1]])

In [ ]:
# root = os.getcwd() + "/ExDark"
# darkDataset = BatchLoader(root, imgHeight=128, imgWidth=128, shuffle=False)
# exDark_train, exDark_val, exDark_test = random_split(darkDataset, [3000, 1800, 2563], generator=torch.Generator().manual_seed(6))
# test_loader = DataLoader(exDark_test, batch_size=1, num_workers=0,
#                             sampler=sampler.SubsetRandomSampler(range(2563)))
# for i, batchdict in enumerate(test_loader):
#     print(batchdict['im_path'])

In [ ]:
# root = os.getcwd() + "/ExDark"
# darkDataset = BatchLoader(root, imgHeight=128, imgWidth=128, shuffle=True)
# print("length of dataset: ", len(darkDataset))
# exdarkLoader = DataLoader(darkDataset, batch_size=1, num_workers=0, shuffle=True)
# for i, dataBatch in enumerate(exdarkLoader):
#     plt.imshow(dataBatch['im'][0].detach().numpy().transpose(1,2,0))
#     plt.show()
#     if i == 30:
#         break

## DataLoader

We use the dataloader to load the train, validation and test sets.

## Training


In [66]:
root = os.getcwd() + '/ExDark'
darkDataset = BatchLoader(root, shuffle=False)
# exDark_train, exDark_val, exDark_test = random_split(darkDataset, [3000, 1800, 2563], generator=torch.Generator().manual_seed(6))
test_loader = DataLoader(darkDataset, batch_size=1, num_workers=0)

print(len(test_loader))

7363


In [52]:
import torchvision.utils as vutils
from models.ImgEnhanceNet import ImgEnhanceNet
from loss_functions import ICE_loss

In [53]:
# # img = plt.imread("./ExDark/Bus/2015_01897.jpg")
# img = plt.imread("./ExDark/Dog/2015_05590.jpg")
# img = img.astype(np.float32) / 255.0
# img = img.transpose(2,0,1)
# img = img[np.newaxis,:]
# device = 'cuda' if torch.cuda.is_available() else 'cpu'
# S = torch.from_numpy(img).to(device)
# icenet = ImgEnhanceNet()
# icenet.load_state_dict(torch.load("./checkpoint_icenet/icenet_190.pth"))

In [54]:
# icenet = icenet.to(device)
# # optimizer = torch.optim.Adam(icenet.parameters(), lr=0.001)
# with torch.no_grad():
#     icenet.eval()
    
#     R, I = icenet(S)
#     R = R.to(device)
#     I = I.to(device)
    
#     loss = ICE_loss(S,R,I)
# #     loss.backward()
# #     optimizer.step()
    
#     plt.imshow(R[0].detach().cpu().numpy().transpose(1,2,0))
#     plt.axis('off')
#     plt.show()
#     vutils.save_image(R.data ,'2015_05590.jpg',
#                           padding=0, normalize = True)
# #         plt.imshow(R[0].detach().cpu().numpy().transpose(1,2,0))
# #         plt.axis('off')
# #         plt.show()

In [55]:
classNames = [i.split("/")[-1] for i in glob.glob(os.getcwd() + "/ExDark" + "/*")]
for cn in classNames:
    os.makedirs("ExDarkIN/" + cn)

FileExistsError: [Errno 17] File exists: 'ExDarkIN/Cat'

In [67]:
model = ImgEnhanceNet()
model.load_state_dict(torch.load("checkpoint_icenet/icenet_200.pth"))

<All keys matched successfully>

In [68]:
import gc
import torchvision.utils as vutils

In [ ]:
newRoot = "ExDarkIN"
test_loss = []
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = model
gc.collect()
torch.cuda.empty_cache()
with torch.no_grad():
    for i, testBatch in enumerate(test_loader):
        model.eval()
        path = testBatch["im_path"][0]
        S = testBatch['im']
        S = S
        Smax, _ = torch.max(S, dim=1, keepdims=True)

        R, I = model(S)
        R = R
        I = I

        Rmax, _ = torch.max(R, dim=1, keepdims=True)
        S_prime = torch.cat([S, Smax], dim=1)
        S_new = torch.cat([S, Rmax], dim=1)
        S_withI = torch.cat([R, I], dim=1)
        
        print("saving image: ", path)
#         plt.imshow(S_prime[0].detach().cpu().numpy().transpose(1,2,0))
#         plt.show()
#         plt.imshow(S_new[0].detach().cpu().numpy().transpose(1,2,0))
#         plt.show()
#         plt.imshow(S_withI[0].detach().cpu().numpy().transpose(1,2,0))
#         plt.show()
        vutils.save_image(R.data , '%s%s' % (newRoot, path), padding=0, normalize = True)
        # iceloss_test = ICE_loss(S, R, I)
        # test_loss.append(iceloss_test)



saving image:  /Cat/2015_03228.jpg
saving image:  /Cat/2015_03649.jpg
saving image:  /Cat/2015_03698.jpg
saving image:  /Cat/2015_03067.jpg
saving image:  /Cat/2015_03406.jpg
saving image:  /Cat/2015_03737.png
saving image:  /Cat/2015_03711.jpg
saving image:  /Cat/2015_03254.jpg
saving image:  /Cat/2015_03635.jpg
saving image:  /Cat/2015_03192.jpg
saving image:  /Cat/2015_03143.jpg
saving image:  /Cat/2015_03285.jpg
saving image:  /Cat/2015_03522.jpg
saving image:  /Cat/2015_03253.jpg
saving image:  /Cat/2015_03195.jpg
saving image:  /Cat/2015_03632.jpg
saving image:  /Cat/2015_03144.jpg
saving image:  /Cat/2015_03525.jpg
saving image:  /Cat/2015_03282.jpg
saving image:  /Cat/2015_03588.jpg
saving image:  /Cat/2015_03138.jpg
saving image:  /Cat/2015_03559.jpg
saving image:  /Cat/2015_03060.jpg
saving image:  /Cat/2015_03401.jpg
saving image:  /Cat/2015_03046.png
saving image:  /Cat/2015_03377.jpg
saving image:  /Cat/2015_03716.jpg
saving image:  /Cat/2015_03473.jpg
saving image:  /Cat/